In [1]:
# Imports
import scipy.io
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# Load the .mat file
data = scipy.io.loadmat('data/condsForSimJ2moMuscles.mat')

# Extract condsForSim struct
conds_for_sim = data['condsForSim']

In [3]:
# Initialize lists to store data for all conditions
go_envelope_all = []
plan_all = []
muscle_all = []

# Get the number of conditions (rows) and delay durations (columns)
num_conditions, num_delays = conds_for_sim.shape

# Loop through each condition and extract data
for i in range(num_conditions):  # 27 conditions
    go_envelope_condition = []
    plan_condition = []
    muscle_condition = []

    for j in range(num_delays):  # 8 delay durations
        condition = conds_for_sim[i, j]

        go_envelope = condition['goEnvelope']
        plan = condition['plan']
        muscle = condition['muscle']

        # Select only muscles 5 and 6 
        selected_muscle_data = muscle[:, [4, 5]]  # Select columns for muscle 5 and 6, which show the nicest multiphasic activity

        go_envelope_condition.append(go_envelope)
        plan_condition.append(plan)
        muscle_condition.append(selected_muscle_data)

    # Stack data for each condition
    go_envelope_all.append(torch.tensor(go_envelope_condition, dtype=torch.float32))
    plan_all.append(torch.tensor(plan_condition, dtype=torch.float32))
    muscle_all.append(torch.tensor(muscle_condition, dtype=torch.float32))

# Stack data for all conditions
go_envelope_tensor = torch.stack(go_envelope_all)
plan_tensor = torch.stack(plan_all)
muscle_tensor = torch.stack(muscle_all)

# Reshape to merge the first two dimensions
go_envelope_tensor = go_envelope_tensor.reshape(-1, *go_envelope_tensor.shape[2:])
plan_tensor = plan_tensor.reshape(-1, *plan_tensor.shape[2:])
muscle_tensor = muscle_tensor.reshape(-1, *muscle_tensor.shape[2:])

# Print shapes
print(f"Go Envelope Tensor Shape: {go_envelope_tensor.shape}")
print(f"Plan Tensor Shape: {plan_tensor.shape}")
print(f"Muscle Tensor Shape: {muscle_tensor.shape}")


Go Envelope Tensor Shape: torch.Size([216, 296, 1])
Plan Tensor Shape: torch.Size([216, 296, 15])
Muscle Tensor Shape: torch.Size([216, 296, 2])


/tmp/ipykernel_303238/1417502240.py:30: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  go_envelope_all.append(torch.tensor(go_envelope_condition, dtype=torch.float32))


In [4]:
# Normalize and standardize a tensor
def normalize_and_standardize(tensor):
    # Normalize: Scale to 0-1 range
    min_val = tensor.min()
    max_val = tensor.max()
    tensor = (tensor - min_val) / (max_val - min_val)

    # Standardize: Shift to zero mean and unit variance
    mean = tensor.mean()
    std = tensor.std()
    tensor = (tensor - mean) / std

    return tensor

# Apply the function to each tensor
normalized_go_envelope_tensor = normalize_and_standardize(go_envelope_tensor)
normalized_plan_tensor = normalize_and_standardize(plan_tensor)
normalized_muscle_tensor = normalize_and_standardize(muscle_tensor)

# Print shapes to confirm
print(f"Normalized Go Envelope Tensor Shape: {normalized_go_envelope_tensor.shape}")
print(f"Normalized Plan Tensor Shape: {normalized_plan_tensor.shape}")
print(f"Normalized Muscle Tensor Shape: {normalized_muscle_tensor.shape}")


Normalized Go Envelope Tensor Shape: torch.Size([216, 296, 1])
Normalized Plan Tensor Shape: torch.Size([216, 296, 15])
Normalized Muscle Tensor Shape: torch.Size([216, 296, 2])


In [5]:
import torch
import torch.nn as nn

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, g, h, tau=50):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.tau = tau  # Time constant
        self.output_linear = nn.Linear(hidden_size, output_size)  # New linear layer

        # Weight initialization
        self.J = nn.Parameter(torch.randn(hidden_size, hidden_size) * (g / torch.sqrt(torch.tensor(hidden_size, dtype=torch.float))))
        self.B = nn.Parameter(torch.randn(hidden_size, input_size) * (h / torch.sqrt(torch.tensor(input_size, dtype=torch.float)))) 
        self.bx = nn.Parameter(torch.zeros(hidden_size))

        # Nonlinearity
        self.nonlinearity = torch.tanh
    
    def forward(self, x, hidden):
        hidden_prev = hidden.clone()
        timestep = self.tau / 10  # Timestep for Euler integration
    
        # Process each time step in the batch
        for t in range(x.size(1)):  # x.size(1) is the temporal dimension
            x_t = x[:, t, :]  # Extract the data for the current time step
    
            for _ in range(int(1 / timestep)):  # Adjust the range for simulation duration
                hidden_update = torch.matmul(self.J, hidden.transpose(0, 1))
                input_update = torch.matmul(self.B, x_t.transpose(0, 1))
                new_hidden = self.nonlinearity(hidden_update + input_update + self.bx.unsqueeze(1))
                new_hidden = new_hidden.transpose(0, 1)
    
                # Euler integration for continuous-time update
                hidden = hidden + (timestep / self.tau) * (-hidden + new_hidden)
    
        # Initialize output tensor
        output = self.output_linear(hidden)  # Apply the output_linear layer
    
        # Calculate the sum of squares of the firing rates and dynamic regularization
        firing_rate_reg = hidden.pow(2).sum()
        dynamic_reg = (hidden - hidden_prev).pow(2).sum().sqrt()

        return output, hidden, firing_rate_reg, dynamic_reg

    def init_hidden(self, batch_size):
        # Initialize hidden state with batch dimension
        return torch.zeros(batch_size, self.hidden_size)

# Hyperparameters
input_size = 16
hidden_size = 300
output_size = 2  # Number of muscles
g = 1.1  # g value
h = 1.0  # h value

# Model instantiation
model = SimpleRNN(input_size, hidden_size, output_size, g, h)


In [6]:
import torch
import torch.nn as nn

class ComplicatedRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, g, h, tau=50):
        super(ComplicatedRNN, self).__init__()
        self.hidden_size = hidden_size
        self.tau = tau  # Time constant

        # Modified weight initialization for a more chaotic regime (g >> 1)
        self.J = nn.Parameter(torch.randn(hidden_size, hidden_size) * (g / torch.sqrt(torch.tensor(hidden_size, dtype=torch.float))))
        self.B = nn.Parameter(torch.randn(hidden_size, input_size) * (h / torch.sqrt(torch.tensor(input_size, dtype=torch.float))))
        self.w = nn.Parameter(torch.zeros(output_size, hidden_size))
        self.bx = nn.Parameter(torch.zeros(hidden_size))
        self.bz = nn.Parameter(torch.zeros(output_size))

        # Nonlinearity remains the same
        self.nonlinearity = torch.tanh
    
    def forward(self, x, hidden):
        timestep = self.tau / 10  # Timestep for Euler integration
        for _ in range(int(1 / timestep)):  # Adjust the range for simulation duration
            hidden_update = torch.matmul(self.J, hidden.T)
            input_update = torch.matmul(self.B, x.T)
            new_hidden = hidden_update + input_update + self.bx.unsqueeze(1)  # Nonlinearity removed
            new_hidden = new_hidden.T
    
            # Euler integration for continuous-time update
            hidden = hidden + (timestep / self.tau) * (-hidden + new_hidden)
        
        output = torch.matmul(self.w, hidden.T) + self.bz.unsqueeze(1)
        output = output.T
    
        return output, hidden

    def init_hidden(self, batch_size):
        # Initialize hidden state with zeros and correct batch dimension
        return torch.zeros(batch_size, self.hidden_size)

input_size = 16
hidden_size = 300
output_size = 2 # Number of muscles
g = 10.0 # Significantly larger g value for complicated model
h = 1.0 # h value remains the same

complicated_model = ComplicatedRNN(input_size, hidden_size, output_size, g, h)

In [7]:
# Convert datasets to TensorDataset and DataLoader for batch processing
from torch.utils.data import TensorDataset, DataLoader

# Concatenate along the third dimension (dim=2)
X_train = torch.cat((normalized_go_envelope_tensor, normalized_plan_tensor), dim=2)

# y_train remains the same
y_train = normalized_muscle_tensor

batch_size = 64  # You can adjust this based on your data size and memory constraints
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)


In [8]:
# Loss Function and Optimizer
criterion = nn.MSELoss()  # MSE Loss for regression tasks
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
def compute_l2_regularization(parameters, alpha):
    l2_reg = sum(p.pow(2.0).sum() for p in parameters)
    return alpha * l2_reg


In [21]:
import matplotlib.pyplot as plt

# Hyperparameters for regularization
alpha = 0.001  # Example value, adjust as necessary
beta = 0.001
gamma = 0.001

# Training Loop
num_epochs = 250  # The number of times the entire dataset is passed through the network
epoch_losses = []  # List to store average loss of each epoch


for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, targets in train_loader:
        batch_size = inputs.size(0)
        h = model.init_hidden(batch_size) # Initialize hidden state with the current batch size
        optimizer.zero_grad()

        total_firing_rate_reg = 0
        total_dynamic_reg = 0

        # Process each time step in the inputs
        for t in range(inputs.shape[1]):
            print(inputs.shape)
            output, h, firing_rate_reg, dynamic_reg = model(inputs[:, t, :], h)
            total_firing_rate_reg += firing_rate_reg
            total_dynamic_reg += dynamic_reg
    
        # Compute loss using the last output and include regularization terms:
        loss = criterion(output, targets[:, -1, :])
        l2_reg = compute_l2_regularization(model.parameters(), alpha)
        rfr_reg = beta * total_firing_rate_reg / inputs.shape[1]  # Average over time steps
        rj_reg = gamma * total_dynamic_reg / inputs.shape[1]  # Average over time steps
    
        total_loss = loss + l2_reg + rfr_reg + rj_reg
        total_loss.backward()
        optimizer.step()
    
        running_loss += total_loss.item()

    avg_loss = running_loss / len(train_loader)
    epoch_losses.append(avg_loss)
    print(f'Epoch {epoch + 1}, Loss: {avg_loss}')

print('Finished Training')

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(range(1, num_epochs + 1), epoch_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.show()


torch.Size([64, 296, 16])
torch.Size([64, 16])


IndexError: too many indices for tensor of dimension 2

In [10]:
import matplotlib.pyplot as plt

# Training Loop
num_epochs = 250  # The number of times the entire dataset is passed through the network
epoch_losses = []  # List to store average loss of each epoch

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        h = model.init_hidden()

        # Process each time step
        for t in range(inputs.shape[1]):  # iterate over time steps
            output, h = complicated_model(inputs[:, t, :], h)

        # Compute loss using the last output (if your task is many-to-one)
        loss = criterion(output, targets[:, -1, :])
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    epoch_losses.append(avg_loss)
    print(f'Epoch {epoch + 1}, Loss: {avg_loss}')

print('Finished Training')

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(range(1, num_epochs + 1), epoch_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.show()

TypeError: SimpleRNN.init_hidden() missing 1 required positional argument: 'batch_size'